In [10]:
import pandas as pd
import numpy as np
import re
import requests
import urllib.request
import json
import os

from time import sleep
from datetime import datetime
from jira import JIRA
from pandas.io.json import json_normalize

In [11]:
# read credentials from environment
username = os.environ['JIRA_USER']
password = os.environ['JIRA_PASS']
jira = JIRA('https://czjira.ness.com', basic_auth=(username, password))

In [31]:
issues = jira.search_issues('filter=33140', maxResults=500)
#issues = jira.search_issues('project=CUZKRUIAN')
# Definování a naplnění polí
key = ([issue.key for issue in issues])
summary = ([issue.fields.summary for issue in issues])
issuetype = ([issue.fields.issuetype for issue in issues])
priority = ([issue.fields.priority for issue in issues])
status = ([issue.fields.status for issue in issues])
updated = ([issue.fields.updated for issue in issues])
reporter = ([issue.fields.reporter.displayName for issue in issues])
due = ([issue.fields.duedate for issue in issues])
labels = ([",".join(issue.fields.labels) for issue in issues])
fixversion = ([issue.fields.fixVersions for issue in issues])

def get_security_if_present(issue):
    if hasattr(issue.fields, 'security'):
        return issue.fields.security.name
    else:
        return "";

sec = ([get_security_if_present(issue) for issue in issues])


In [32]:
#definování názvů sloupců 
tickety = {'type': issuetype,
            'key': key,
            'summary': summary,
            'priority': priority,
            'status': status,
            'updated': updated,
            'due': due,
            'labels': labels,
            'fixversion': fixversion,
            'security': sec}
print(tickety)

{'type': [<JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Chyba', id='1'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Úkol', id='3'>, <JIRA IssueType: name='Úkol', id='3'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Nová vlastnost', id='2'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Chyba', id='1'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType: name='Vedlejší úkol', id='6'>, <JIRA IssueType

In [33]:
# pandas
DF = pd.DataFrame.from_dict(tickety)

In [34]:
#srovná sloupce do pořadí 
JT = DF[["type","key","summary","priority","status","updated","due","labels","fixversion","security"]]

In [35]:
#vybere vše ve sloupci summary, kde je obsažené slovo clone
#clone = JT[JT['summary'].str.contains('CLONE')]

In [36]:
#vybere vše mimo ...
JT2 = JT[~JT['summary'].str.contains('CLONE')]
JT3 = JT2[~JT2['key'].str.contains('RUIANADM-19')]
JT4 = JT3[~JT3['key'].str.contains('RUIANADM-1086')]
#JT5 = JT4[JT4['labels'].loc.contains('Long_task')] - tady mám problém, budu muset vymazat [] ještě před pandasem :) 

In [37]:
#seřazení 
JT4.sort_values(by = ['security','updated'], ascending=[True,False])

,type,key,summary,priority,status,updated,due,labels,fixversion,security
1,Chyba,RUIANADM-1337,Pomalý první přístup na SharePoint: https://cz...,Nízká,Probíhá řešení,2018-03-19T12:01:12.000+0100,None,,[],
0,Vedlejší úkol,RUIANADM-1339,Support report 2018-02,Vysoká,Probíhá řešení,2018-03-14T09:19:38.000+0100,None,,[],
4,Vedlejší úkol,RUIANADM-1310,Support report 2018-01,Vysoká,Probíhá řešení,2018-03-14T09:17:53.000+0100,2017-07-18,,[],
2,Vedlejší úkol,RUIANADM-1323,RUIAN-2.3.2 - úprava filtrů a dashboardů,Vysoká,Znovu otevřeno,2018-02-19T13:10:35.000+0100,None,,[],
20,Vedlejší úkol,RUIANADM-1194,[RUIAN-2.3] Akceptace,Nízká,Probíhá řešení,2018-02-19T10:06:08.000+0100,None,,[],
89,Chyba,RUIANADM-775,Vykazování práce - řídící a režijní činnosti,Nízká,Otevřeno,2018-02-09T00:57:03.000+0100,None,"Internal,Long_task",[],
7,Úkol,RUIANADM-1303,Objednávka služeb VersaSystems na rok 2018,Nízká,Probíhá řešení,2018-02-08T22:25:19.000+0100,None,,[],
6,Úkol,RUIANADM-1305,Objednávka služeb AutoCont na rok 2018,Nízká,Probíhá řešení,2018-02-08T21:57:38.000+0100,None,,[],
16,Nová vlastnost,RUIANADM-1221,Dodávka RUIAN-2.4,Vysoká,Otevřeno,2018-02-07T12:56:53.000+0100,2018-11-30,Long_task,[],
3,Vedlejší úkol,RUIANADM-1314,Zastavená komunikační fronta ISUI na TEST3,Vysoká,Otevřeno,2018-01-23T17:10:35.000+0100,None,,[],


In [10]:
#počet ticketů
print ("CUZKRUIANADM: ", [len(JT4[JT4['key'].str.contains('RUIANADM-')])])
print ("CUZKRUIAN: ", [len(JT4[JT4['key'].str.contains('RUIAN-')])])
print ("PATCHMAN: ", [len(JT4[JT4['key'].str.contains('PATCHMAN')])])
print ("Celkem: ", [len(JT4)])

CUZKRUIANADM:  [85]
CUZKRUIAN:  [113]
PATCHMAN:  [1]
Celkem:  [199]


In [11]:
#jelikož jsem nepřišla na to, jak se v Jiře jmenuje "Security level", 
#tak není seřazeno dle Interní/externí" a tento sloupec také chybí

In [18]:
JT4.to_csv('2018-04-05.csv', encoding='utf-8', index=False)